<a href="https://colab.research.google.com/github/FlyAIBox/AIAgent101/blob/main/06-agent-evaluation/langfuse/evaluation_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
在 Langfuse 上运行 LangChain 评测

本指南演示如何使用基于模型的评测，自动化评估 Langfuse 中线上产出的 LLM 完成结果。示例使用 LangChain，亦可迁移到其他库；最佳选择取决于具体场景。

本指南分三步：
1. 从 Langfuse 获取线上存储的 `generations`
2. 使用 LangChain 对这些 `generations` 进行评测
3. 将结果作为 `scores` 回灌到 Langfuse


----
还未使用 Langfuse？通过采集 LLM 事件，[立即开始](https://langfuse.com/docs/get-started)。

### 环境准备

先用 pip 安装 Langfuse 与 LangChain，然后设置环境变量。

In [1]:
%pip install langfuse==3.3.0 langchain==0.3.27 langchain-openai==0.3.31

In [4]:
# 环境变量配置
# 设置OpenAI API密钥，这是使用OpenAI模型所必需的
import os, getpass

def _set_env(var: str):
    """
    安全地设置环境变量
    如果环境变量不存在，会提示用户输入
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 设置OpenAI API密钥
# 您需要从 https://platform.openai.com/api-keys 获取API密钥
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")


# 在项目设置页获取密钥：https://cloud.langfuse.com,
# os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-2cf6d992-461d-4b3c-a47e-96285c1e5526@FLYAIBOX"
# os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-55c2fa56-c913-44be-916b-85ba498dcb8a@FLYAIBOX"
# PUBLIC KEY
_set_env("LANGFUSE_PUBLIC_KEY")
# SECRET KEY
_set_env("LANGFUSE_SECRET_KEY")
# 🇪🇺 欧盟区域(推荐) https://cloud.langfuse.com
# 🇺🇸 美国区域 https://us.cloud.langfuse.com
_set_env("LANGFUSE_HOST")

LANGFUSE_PUBLIC_KEY: ··········
LANGFUSE_SECRET_KEY: ··········
LANGFUSE_HOST: ··········


In [7]:
import os

# 设置用于评估的LLM模型名称，这里使用gpt-3.5-turbo-instruct
os.environ['EVAL_MODEL'] = "gpt-3.5-turbo-instruct"

# LangChain 评测类型
# 定义一个字典来指定要执行哪些LangChain评测
EVAL_TYPES={
    "hallucination": True, # 是否评估幻觉（生成的内容是否包含输入或参考中不存在的信息）
    "conciseness": True, # 是否评估简洁性（生成的内容是否简洁明了）
    "relevance": True, # 是否评估相关性（生成的内容是否与输入相关）
    "coherence": True, # 是否评估连贯性（生成的内容是否逻辑连贯）
    "harmfulness": True, # 是否评估有害性（生成的内容是否包含有害信息）
    "maliciousness": True, # 是否评估恶意性（生成的内容是否包含恶意信息）
    "helpfulness": True, # 是否评估有用性（生成的内容是否对用户有用）
    "controversiality": True, # 是否评估争议性（生成的内容是否包含争议性观点）
    "misogyny": True, # 是否评估性别歧视症（生成的内容是否包含性别歧视症观点）
    "criminality": True, # 是否评估犯罪性（生成的内容是否包含鼓励犯罪的信息）
    "insensitivity": True # 是否评估不敏感性（生成的内容是否包含不敏感信息）
}

初始化 Langfuse Python SDK，更多信息见[此处](https://langfuse.com/docs/sdk/python#1-installation)。

In [8]:
from langfuse import get_client

langfuse = get_client()

# 验证连接
if langfuse.auth_check():
    print("Langfuse 客户端已通过认证，准备就绪！")
else:
    print("认证失败。请检查凭据与主机配置。")

Langfuse 客户端已通过认证，准备就绪！


### 拉取数据

根据 `name` 从 Langfuse 载入所有 `generations`，此处示例为 `OpenAI`。在 Langfuse 中，`name` 用于标识应用内不同类型的生成。将其替换为你需要评测的名称。

关于在写入 LLM Generation 时如何设置 `name`，参见[文档](https://langfuse.com/docs/sdk/python#generation)。

In [6]:
def fetch_all_pages(name=None, user_id = None, limit=50):
    page = 1
    all_data = []

    while True:
        response = langfuse.api.trace.list(name=name, limit=limit, user_id=user_id, page=page)
        if not response.data:
            break

        all_data.extend(response.data)
        page += 1

    return all_data

In [7]:
generations = fetch_all_pages(user_id='user_123')

In [8]:
print(generations)

[]


In [9]:
generations[0].id

IndexError: list index out of range

### 定义评测函数

本节基于 `EVAL_TYPES` 定义 LangChain 评测器；其中“幻觉”（hallucination）需要单独函数。关于 LangChain 评测的更多信息见[此处](https://python.langchain.com/docs/guides/evaluation/string/criteria_eval_chain)。

In [ ]:
# 导入 LangChain 评测器和 OpenAI 模型
from langchain.evaluation import load_evaluator
from langchain_openai import OpenAI
from langchain.evaluation.criteria import LabeledCriteriaEvalChain

# 定义一个函数，根据给定的评测标准键（key）获取对应的 LangChain 评测器
def get_evaluator_for_key(key: str):
  # 初始化一个 OpenAI 模型，设置 temperature=0 以获得更稳定的输出
  # 使用环境变量 'EVAL_MODEL' 指定模型名称
  llm = OpenAI(temperature=0, model=os.environ.get('EVAL_MODEL'))
  # 使用 load_evaluator 函数加载指定标准的评测器
  return load_evaluator("criteria", criteria=key, llm=llm)

# 定义一个函数，获取用于评估“幻觉”（hallucination）的评测器
def get_hallucination_eval():
  # 定义“幻觉”的评测标准
  criteria = {
    "hallucination": (
      "Does this submission contain information"
      " not present in the input or reference?" # 检查输出是否包含输入或参考中不存在的信息
    ),
  }
  # 初始化一个 OpenAI 模型
  llm = OpenAI(temperature=0, model=os.environ.get('EVAL_MODEL'))

  # 使用 LabeledCriteriaEvalChain.from_llm 创建一个基于 LLM 的评测链
  return LabeledCriteriaEvalChain.from_llm(
      llm=llm,
      criteria=criteria, # 使用上面定义的“幻觉”标准
  )

### 执行评测

下面将对上面载入的每个 `Generation` 执行评测。每个得分将通过 [`langfuse.score()`](https://langfuse.com/docs/scores) 写回 Langfuse。


In [ ]:
def execute_eval_and_score():

  for generation in generations:
    criteria = [key for key, value in EVAL_TYPES.items() if value and key != "hallucination"]

    for criterion in criteria:
      eval_result = get_evaluator_for_key(criterion).evaluate_strings(
          prediction=generation.output,
          input=generation.input,
      )
      print(eval_result)

      langfuse.create_score(name=criterion, trace_id=generation.id, observation_id=generation.id, value=eval_result["score"], comment=eval_result['reasoning'])

execute_eval_and_score()


In [ ]:
# 幻觉（hallucination）

def eval_hallucination():

  chain = get_hallucination_eval()

  for generation in generations:
    eval_result = chain.evaluate_strings(
      prediction=generation.output,
      input=generation.input,
      reference=generation.input
    )
    print(eval_result)
    if eval_result is not None and eval_result["score"] is not None and eval_result["reasoning"] is not None:
      langfuse.create_score(name='hallucination', trace_id=generation.id, observation_id=generation.id, value=eval_result["score"], comment=eval_result['reasoning'])


In [ ]:
if EVAL_TYPES.get("hallucination") == True:
  eval_hallucination()

In [ ]:
# SDK 为异步实现，请确保等待所有请求完成
langfuse.flush()

### 在 Langfuse 中查看分数

在 Langfuse 界面中，你可以按 `Scores` 过滤 Traces，并查看每条的详细信息。你也可以通过 Langfuse Analytics 分析新提示词版本或应用发布对这些分数的影响。

![Trace 示例](https://langfuse.com/images/docs/trace-conciseness-score.jpg)
_包含简洁性（conciseness）分数的示例 Trace_
